In [1]:
import os 
import sys 

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

from functools import cache 
import numpy as np 
import pandas as pd 
import altair as alt 
from IPython.display import clear_output
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../.env')

print(os.environ['SUBGRAPH_URL'])

from utils_notebook.utils import ddf, load_subgraph, remove_prefix
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    stack_order_expr, 
    wide_to_longwide, 
    chart_stack_area_overlay_line_timeseries,
)
from utils_notebook.queries import adjust_precision, QueryManager
from utils_notebook.testing import validate_season_series
from utils_notebook.css import css_tooltip_timeseries_multi_colored
from utils_notebook.queries import QueryManager

https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk


In [2]:
sg: Subgrounds
bs: Subgraph
sg, bs = load_subgraph()

In [3]:
q = QueryManager(sg, bs) 

In [4]:
pool = bs.Query.metapoolOracles(first=100000, orderBy="season", orderDirection="asc")
df = sg.query_df(
    [
        pool.balanceA, 
        pool.balanceB, 
        pool.season, 
        pool.deltaB, 
        pool.timestamp, 
    ],
    pagination_strategy=ShallowStrategy
)
df = remove_prefix(df, "metapoolOracles_")
df.balanceA /= 10**6
df.balanceB /= 10**18
df.deltaB /= 10**6

In [5]:
# Reverse engineer pool reserves from quantites used in TWAP calculation. 
df['diff_a'] = (df.balanceA - df.balanceA.shift(1))
df['diff_b'] = (df.balanceB - df.balanceB.shift(1))
df['diff_timestamp'] = (df.timestamp - df.timestamp.shift(1))
df['reserves_3crv'] = df['diff_a'] / df.diff_timestamp
df['reserves_bean'] = df['diff_b'] / df.diff_timestamp
assert df.season.min() == 6076
df = df.iloc[1:,]
df.head()

,balanceA,balanceB,season,deltaB,timestamp,diff_a,diff_b,diff_timestamp,reserves_3crv,reserves_bean
1,1823002386736.906738,2030942247135.286865,6077,685226.464263,1659812464,46790827319.678223,50589125510.827393,3533.0,13243936.40523,14319027.883053
2,1869934826665.250244,2081743078278.517822,6078,694584.048577,1659816002,46932439928.343506,50800831143.230957,3538.0,13265245.881386,14358629.492151
3,1917706461458.546631,2133501113442.06958,6079,701977.691800,1659819600,47771634793.296387,51758035163.551758,3598.0,13277274.817481,14385223.780865
4,1965608261630.781006,2186128547794.177979,6080,803240.828322,1659823204,47901800172.234375,52627434352.108398,3604.0,13291287.506169,14602506.756967
5,2013881826521.260254,2239501123147.967285,6081,849932.384158,1659826833,48273564890.479248,53372575353.789307,3629.0,13302167.23353,14707240.38407


In [6]:
df_szns = q.query_seasons(extra_cols=['price'], where={"season_gte": 6074})[['season', 'price']]
df_szns = df_szns.rename(columns={"price": "price_bean"})
df_szns.head()

,season,price_bean
0,6074,1.022000
1,6075,1.070000
2,6076,1.050748
3,6077,1.051615
4,6078,1.051964


In [7]:
df = df.merge(df_szns, how="left", on="season")
df.head()

,balanceA,balanceB,season,deltaB,timestamp,diff_a,diff_b,diff_timestamp,reserves_3crv,reserves_bean,price_bean
0,1823002386736.906738,2030942247135.286865,6077,685226.464263,1659812464,46790827319.678223,50589125510.827393,3533.0,13243936.40523,14319027.883053,1.051615
1,1869934826665.250244,2081743078278.517822,6078,694584.048577,1659816002,46932439928.343506,50800831143.230957,3538.0,13265245.881386,14358629.492151,1.051964
2,1917706461458.546631,2133501113442.06958,6079,701977.691800,1659819600,47771634793.296387,51758035163.551758,3598.0,13277274.817481,14385223.780865,1.052462
3,1965608261630.781006,2186128547794.177979,6080,803240.828322,1659823204,47901800172.234375,52627434352.108398,3604.0,13291287.506169,14602506.756967,1.062113
4,2013881826521.260254,2239501123147.967285,6081,849932.384158,1659826833,48273564890.479248,53372575353.789307,3629.0,13302167.23353,14707240.38407,1.063713


In [8]:
# Approximation of pool TVL in $, since we can't compute this exactly without the price of 3Crv 
df['pool_tvl_usd'] = 2 * df.price_bean * (df.reserves_bean - df.deltaB)
df['bean_fraction'] = (df.price_bean * df.reserves_bean) / df.pool_tvl_usd
df['3crv_fraction'] = 1 - df.bean_fraction
df.tail()

,balanceA,balanceB,season,deltaB,timestamp,diff_a,diff_b,diff_timestamp,reserves_3crv,reserves_bean,price_bean,pool_tvl_usd,bean_fraction,3crv_fraction
1672,93999374527910.375,91871577961655.578125,7749,-3547.824471,1665831611,50568295499.703125,49449791265.46875,3600.0,14046748.749918,13736053.129297,0.999747,27472249.669453,0.499871,0.500129
1673,94049942823410.078125,91921027752921.0625,7750,-3547.795945,1665835211,50568295499.703125,49449791265.484375,3600.0,14046748.749918,13736053.129301,0.999747,27472249.612424,0.499871,0.500129
1674,94100511118909.78125,91970477544186.53125,7751,-3547.730689,1665838811,50568295499.703125,49449791265.46875,3600.0,14046748.749918,13736053.129297,0.999747,27472249.481937,0.499871,0.500129
1675,94151079414409.5,92019927335452.0,7752,-3546.594580,1665842411,50568295499.71875,49449791265.46875,3600.0,14046748.749922,13736053.129297,0.999747,27472247.210293,0.499871,0.500129
1676,94201647709909.203125,92069377126717.484375,7753,-3546.487409,1665846011,50568295499.703125,49449791265.484375,3600.0,14046748.749918,13736053.129301,0.999747,27472246.996014,0.499871,0.500129


In [9]:
base = alt.Chart(df[['timestamp', 'reserves_3crv', 'reserves_bean', 'deltaB']])
left = (
    base
    .transform_filter("datum.key != 'deltaB'")
    .transform_fold(['reserves_3crv', 'reserves_bean'])
    .mark_line()
    .encode(
        x="timestamp:T", 
        y="value:Q", 
        color="key:O", 
    )
)
right = (
    base
    .mark_line()
    .encode(
        x="timestamp:T", 
        y="deltaB:Q", 
    )
)
left | right

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [10]:
base = alt.Chart(df[['timestamp', 'bean_fraction', '3crv_fraction']])
c = (
    base
    .transform_fold(['bean_fraction', '3crv_fraction'])
    .mark_line()
    .encode(
        x="timestamp:T", 
        y="value:Q", 
        color="key:O", 
    )
)
c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [11]:
"""
Pool Level Metrics 
    reserves 
        bean 
        3crv 
    balance 
        bean 
        3crv 
    liquidity (USD)
    volume (USD) (Can't get this through subgraph) 

Metrics Across Pools 


"""

"\nPool Level Metrics \n    reserves \n        bean \n        3crv \n    balance \n        bean \n        3crv \n    liquidity (USD)\n    volume (USD) (Can't get this through subgraph) \n\nMetrics Across Pools \n\n\n"